In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
from dataset import iterable_dataset
from models.music_generation_gru import MusicGRU
import os
from utils import training_rnn, data_reading
import torch
from torch.utils.data import DataLoader
import pickle
import matplotlib.pyplot as plt
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
PROJECT_DIR ="D:/522 project"
PPR_DIR = "lpd_5_pianoroll"
SEQ_LEN = 48
RESOLUTION = 12
TRAIN_SPLIT = 0.8
HIDDEN_SIZE = 512
BATCH_SIZE = 1
N_LAYERS = 6
ALPHA = 5
LR = 0.0001
LR_LAMBDA = 1
EPOCH = 20
AUTOSAVE = True

TRAIN_BUFFER_SIZE = 1000
TRAIN_FILES_TO_READ = 1000

TEST_BUFFER_SIZE = 100
TEST_FILE_TO_READ = 100

SCALER_DICT_SAVE_PATH = "./gru_saving/gru_piano_scaler_dict.pkl"
TRACK_ORDER_SAVE_PATH = "./gru_saving/gru_piano_track_order.pkl"
SAVE_ON_EPOCH = 1
PATH = os.path.join(PROJECT_DIR, PPR_DIR)
INSTRUMENT = ["Piano"]
all_file_paths = data_reading.read_all_npzs(PATH, True)
train_file_paths = all_file_paths[:round(len(all_file_paths)*TRAIN_SPLIT)]
test_file_paths = all_file_paths[round(len(all_file_paths)*TRAIN_SPLIT):]
train_piano_dataset = iterable_dataset.IterableSongDataset(train_file_paths, SEQ_LEN, RESOLUTION,  INSTRUMENT, 
                                          method="one_step_window", normalize=False, binarize=True,
                                          buffer_size=TRAIN_BUFFER_SIZE, files_to_read=TRAIN_FILES_TO_READ)
train_piano_loader = DataLoader(train_piano_dataset, batch_size = BATCH_SIZE, drop_last=True)
test_piano_dataset = iterable_dataset.IterableSongDataset(test_file_paths, SEQ_LEN, RESOLUTION, INSTRUMENT, 
                                         method="one_step_window", normalize=False, binarize=True,
                                         buffer_size=TEST_BUFFER_SIZE, files_to_read=TEST_FILE_TO_READ)
test_piano_loader = DataLoader(test_piano_dataset, batch_size = BATCH_SIZE, drop_last=True)
model = MusicGRU(train_piano_dataset.get_track_order(), HIDDEN_SIZE, BATCH_SIZE, N_LAYERS, DEVICE).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda = lambda epoch: LR_LAMBDA ** epoch)
criterion = torch.nn.BCEWithLogitsLoss()


In [ ]:
train_losses, test_losses = training_rnn.training_rnn(model, optimizer, scheduler, criterion, train_piano_loader,
             test_piano_loader, EPOCH, DEVICE, BATCH_SIZE,  ALPHA, AUTOSAVE, SAVE_ON_EPOCH)


[0.0001]
reloading
start test
reloading
Epoch 1, Train Loss: 0.049541068039186155, Test Loss: 0.046997425975713256, Time: 2022-04-26 19:38:49.673455
[0.0001]
reloading
start test
reloading
Epoch 2, Train Loss: 0.02634300189938954, Test Loss: 0.027668908345455854, Time: 2022-04-26 20:04:14.445948
[0.0001]
reloading
start test
reloading
Epoch 3, Train Loss: 0.02048646420508356, Test Loss: 0.022161790301494608, Time: 2022-04-26 20:29:46.941446
[0.0001]
reloading
start test
reloading
Epoch 4, Train Loss: 0.017450066418834693, Test Loss: 0.018974936642897, Time: 2022-04-26 20:55:21.528946
[0.0001]
reloading
start test
reloading
Epoch 5, Train Loss: 0.01547469234875736, Test Loss: 0.017552587803662013, Time: 2022-04-26 21:21:47.165659
[0.0001]
reloading
start test
reloading
Epoch 6, Train Loss: 0.014228338043177552, Test Loss: 0.016521616400841016, Time: 2022-04-26 21:48:05.243455
[0.0001]
reloading
start test
reloading
Epoch 7, Train Loss: 0.013512567514562937, Test Loss: 0.0158410136159138

In [ ]:

with open(TRACK_ORDER_SAVE_PATH, "wb") as file:
    pickle.dump(train_piano_dataset.get_track_order(), file)

In [ ]:
plt.rcParams["figure.figsize"] = (20,20)
plt.plot(train_losses, label = 'Train Loss')
plt.plot(test_losses, label = 'Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
